# 01 Build Macro Monthly Panel

Align mixed-frequency series to a month-end panel.


## Table of Contents
- [Load series](#load-series)
- [Month-end alignment](#month-end-alignment)
- [Missingness](#missingness)
- [Save processed panel](#save-processed-panel)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## Prerequisites (Quick Self-Check)
- Completed Part 00 (foundations) or equivalent time-series basics.
- FRED API key set (`FRED_API_KEY`) for real data (sample data works offline).

## What You Will Produce
- data/processed/panel_monthly.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/01_data/01_build_macro_monthly_panel.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/01_data/01_build_macro_monthly_panel.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Build a clean **month-end** panel of predictors.

### Why this matters
Most economic indicators are not recorded at the same frequency.
Before modeling, you must decide:
- the timeline you will predict on (monthly vs quarterly)
- how to align mixed-frequency indicators onto that timeline

In this notebook, you standardize everything to **month-end**.



## Primer: pandas time series essentials (indexing, resampling, lags)

Most “mysterious bugs” in time series work come from index and alignment mistakes. This primer gives you the minimum patterns to avoid them.

### 1) DatetimeIndex (the first thing to verify)

Most time-series operations assume a `DatetimeIndex`:

```python
import pandas as pd

df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)
```

**Expected output / sanity checks**
- `df.index.min(), df.index.max()` look reasonable
- `df.index.is_monotonic_increasing` is `True`

### 2) Resampling (frequency alignment)

Resampling converts one frequency to another. Choose the aggregation rule intentionally.

```python
# month-end last value (end-of-period)
df_me_last = df.resample("ME").last()

# month-end mean (average-of-period)
df_me_mean = df.resample("ME").mean()

# quarter-end mean
df_q_mean = df.resample("QE").mean()
```

**Interpretation matters**
- `.last()` treats end-of-period value as “the period’s value.”
- `.mean()` treats the period average as “the period’s value.”

### 3) Alignment and merging

When joining series, always check missingness after the join:

```python
merged = df1.join(df2, how="outer").sort_index()
print(merged.isna().sum().sort_values(ascending=False).head(10))
```

### 4) Lags and rolling windows (watch for leakage!)

```python
# lag 1 period (past-only)
df["x_lag1"] = df["x"].shift(1)

# rolling mean using past values ending at t
df["x_roll12"] = df["x"].rolling(12).mean()
```

**Leakage pitfalls**
- `shift(-1)` uses the future.
- `rolling(..., center=True)` uses the future.

### 5) A quick workflow you should repeat

1) Set and verify DatetimeIndex.
2) Resample intentionally (mean vs last).
3) Join and inspect missingness.
4) Add lags/rolls (past-only).
5) `dropna()` to build a clean modeling table.


<a id="load-series"></a>
## Load series

### Goal
Load a macro indicator panel.

Options:
1) If you completed `00_fred_api_and_caching`, load raw series from `data/raw/fred/` JSON files.
2) Otherwise, load the offline sample from `data/sample/panel_monthly_sample.csv`.



### Your Turn (1): Try to load from cached JSON (preferred)


In [ ]:
import pandas as pd
from src import data as data_utils
from src import fred_api

# TODO: Choose the same series_ids you used before.
series_ids = ['UNRATE', 'FEDFUNDS', 'CPIAUCSL', 'INDPRO', 'RSAFS', 'T10Y2Y']

raw_dir = RAW_DIR / 'fred'

# TODO: If JSON files exist, load them and build a raw panel.
frames = []
for sid in series_ids:
    path = raw_dir / f'{sid}.json'
    if not path.exists():
        continue
    payload = data_utils.load_json(path)
    frames.append(fred_api.observations_to_frame(payload, sid))

panel_raw = pd.concat(frames, axis=1).sort_index() if frames else None
panel_raw.head() if panel_raw is not None else None



### Your Turn (2): Fallback to sample if needed


In [ ]:
import pandas as pd

# TODO: If panel_raw is None, load the sample.
if panel_raw is None:
    panel_raw = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

panel_raw.head()



<a id="month-end-alignment"></a>
## Month-end alignment

### Goal
Convert the panel to **month-end** index and decide how to handle series that update more frequently.

In later notebooks we will aggregate monthly -> quarterly; for now, everything becomes monthly.



### Your Turn (1): Ensure month-end index


In [ ]:
from src import features

# TODO: If panel_raw is already month-end, verify it.
# If it is daily (or mixed), resample to month-end.
# Hint: features.to_monthly(panel_raw)
panel_me = ...

panel_me.head()



### Your Turn (2): Compare before/after resampling


In [ ]:
# TODO: Print index frequency guesses before and after.
# TODO: Print how many rows you have before/after.
...



<a id="missingness"></a>
## Missingness

### Goal
Inspect missing values and choose a strategy.

In macro panels, a common approach is:
- forward-fill within a series after resampling
- then drop early rows that are still missing because the series starts later



### Your Turn (1): Missingness report


In [ ]:
# TODO: Print missing values per column and as a percent
na_counts = panel_me.isna().sum().sort_values(ascending=False)
na_pct = (na_counts / len(panel_me)).round(3)
pd.DataFrame({'na': na_counts, 'na_pct': na_pct})



### Your Turn (2): Decide and apply a strategy


In [ ]:
# TODO: Choose a missingness strategy.
# Default suggestion:
# - forward fill
# - drop remaining NaNs
panel_filled = panel_me.ffill()
panel_clean = panel_filled.dropna().copy()

panel_clean.head()



<a id="save-processed-panel"></a>
## Save processed panel

### Goal
Write your month-end panel to `data/processed/panel_monthly.csv`.



### Your Turn (1): Save


In [ ]:
out_path = PROCESSED_DIR / 'panel_monthly.csv'
out_path.parent.mkdir(parents=True, exist_ok=True)

# TODO: Save panel_clean to CSV
...



### Your Turn (2): Load back and validate


In [ ]:
import pandas as pd

# TODO: Load the saved file and confirm it matches your in-memory data.
panel_check = pd.read_csv(PROCESSED_DIR / 'panel_monthly.csv', index_col=0, parse_dates=True)
assert panel_check.shape == panel_clean.shape
assert panel_check.index.is_monotonic_increasing
panel_check.head()



### Checkpoint


In [ ]:
# TODO: Assert no missing values remain and index is month-end-ish.
assert not panel_clean.isna().any().any()
assert panel_clean.shape[0] > 100
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: data/processed/panel_monthly.csv
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Load series</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_build_macro_monthly_panel — Load series
import pandas as pd
from src import data as data_utils
from src import fred_api

raw_dir = RAW_DIR / 'fred'
series_ids = ['UNRATE', 'FEDFUNDS', 'CPIAUCSL', 'INDPRO', 'RSAFS', 'T10Y2Y']

frames = []
for sid in series_ids:
    payload = data_utils.load_json(raw_dir / f'{sid}.json') if (raw_dir / f'{sid}.json').exists() else None
    if payload is None:
        # Offline fallback
        panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)
        break
    frames.append(fred_api.observations_to_frame(payload, sid))
else:
    panel = pd.concat(frames, axis=1).sort_index()

panel.head()
```

</details>

<details><summary>Solution: Month-end alignment</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_build_macro_monthly_panel — Month-end alignment
# Convert to month-end panel (ME) and forward-fill.
panel_monthly = panel.resample('ME').last().ffill()
panel_monthly.tail()
```

</details>

<details><summary>Solution: Missingness</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_build_macro_monthly_panel — Missingness
missing = panel_monthly.isna().sum().sort_values(ascending=False)
missing.head(10)
```

</details>

<details><summary>Solution: Save processed panel</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_build_macro_monthly_panel — Save processed panel
from src import data as data_utils
data_utils.save_csv(panel_monthly, PROCESSED_DIR / 'panel_monthly.csv')
print('saved', PROCESSED_DIR / 'panel_monthly.csv')
```

</details>

